Prepare the Data

In [2]:
import pickle
docs = pickle.load(open('raw_docs_filtered.pkl', 'rb'))

In [3]:
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize

def split_in_segments(text):
    tokens = 0
    mystring = list()
    segments = []
    for sent in sent_tokenize(text):
        newtokens = len(sent.split())
        tokens += newtokens
        mystring.append(str(sent).strip())
        if tokens > 22:
            segments.append(" ".join(mystring))
            mystring = []
            tokens = 0
    if mystring:
        segments.append(" ".join(mystring))
    return segments
texts = []
for i, row in docs.iterrows():
    texts.extend(split_in_segments(row['document_text']))

In [24]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from scipy.special import softmax

MODEL = f"soleimanian/financial-roberta-large-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

text = [""" While the equity market had a positive
            month, mainly driven by a handful of IT, high growth stocks,
            (i.e. Nvidia).""",
        """
        the bond market ended in negative territory, as
            rates’ cuts expectations moved further out into the year.
        """]

encoded_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

In [25]:
softmax(output[0][0].detach().numpy())

array([0.00170091, 0.19778153, 0.80051756], dtype=float32)

In [26]:
df = pd.DataFrame([softmax(x.detach().numpy()) for x in output[0]], columns=['negative', 'neutral', 'positive'])
df.head()

,negative,neutral,positive
0,0.001701,0.197782,0.800518
1,0.998345,0.001080,0.000575


In [28]:
df['text'] = text
df.head() 

,negative,neutral,positive,text
0,0.001701,0.197782,0.800518,While the equity market had a positive\n ...
1,0.998345,0.001080,0.000575,\n the bond market ended in negative te...
